<a href="https://colab.research.google.com/github/Akshayabalaji23/Data-Pipeline-for-Amazon-Customer-Review-Analysis/blob/main/Data_pipeline_for_Amazon_Customer_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from datetime import datetime

# ---------- Task 1: Data Extraction ----------
file_path = "aws_data - Sheet1 (3).csv"  # Replace with actual path if needed
df = pd.read_csv(file_path)

# ---------- Task 2: Data Cleaning ----------
df = df.drop_duplicates()
df = df.dropna()

# Correct data types
df['review_date'] = pd.to_datetime(df['review_date'], errors='coerce')
df['star_rating'] = df['star_rating'].astype(int)
df['helpful_votes'] = df['helpful_votes'].astype(int)
df['total_votes'] = df['total_votes'].astype(int)

# Standardize fields
df['marketplace'] = df['marketplace'].str.upper().str.strip()
df['product_category'] = df['product_category'].str.title().str.strip()

# ---------- Task 3: Data Transformation ----------
# Normalize text fields
df['review_headline'] = df['review_headline'].str.lower().str.strip()
df['review_body'] = df['review_body'].str.lower().str.strip()

# Create new feature: review_month
df['review_month'] = df['review_date'].dt.to_period('M').astype(str)

# ---------- Task 4: Load into Remote SQL Database ----------

# Example using PostgreSQL (replace with your credentials)
host = "your-db-host"
port = "5432"
database = "your_database"
username = "your_username"
password = "your_password"

# Construct connection URL
connection_url = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_url)

# Table name
table_name = "amazon_reviews"

# Load to SQL
df.to_sql(table_name, engine, if_exists='replace', index=False)

print(f"✅ Loaded {len(df)} records into '{table_name}' table successfully.")


In [ ]:
-- Task 5.1: Top 10 Products with the Most Reviews
SELECT product_title, COUNT(*) AS review_count
FROM amazon_reviews
GROUP BY product_title
ORDER BY review_count DESC
LIMIT 10;

-- Task 5.2: Average Review Ratings Per Month for Each Product
SELECT product_title, review_month, AVG(star_rating) AS avg_rating
FROM amazon_reviews
GROUP BY product_title, review_month
ORDER BY product_title, review_month;

-- Task 5.3: Total Number of Votes Per Product Category
SELECT product_category, SUM(total_votes) AS total_votes
FROM amazon_reviews
GROUP BY product_category
ORDER BY total_votes DESC;

-- Task 5.4: Products with the Word 'Awful' Occurring Most
SELECT product_title, COUNT(*) AS awful_mentions
FROM amazon_reviews
WHERE review_body LIKE '%awful%'
GROUP BY product_title
ORDER BY awful_mentions DESC
LIMIT 10;

-- Task 5.5: Products with the Word 'Awesome' Occurring Most
SELECT product_title, COUNT(*) AS awesome_mentions
FROM amazon_reviews
WHERE review_body LIKE '%awesome%'
GROUP BY product_title
ORDER BY awesome_mentions DESC
LIMIT 10;

-- Task 5.6: Most Controversial Reviews (High Votes, Low Helpful %)
SELECT review_id, product_title, total_votes, helpful_votes,
       ROUND(100.0 * helpful_votes / NULLIF(total_votes, 0), 2) AS helpful_percentage
FROM amazon_reviews
WHERE total_votes >= 10
ORDER BY helpful_percentage ASC, total_votes DESC
LIMIT 10;

-- Task 5.7: Most Commonly Reviewed Product Per Year
SELECT EXTRACT(YEAR FROM review_date) AS year, product_title, COUNT(*) AS review_count
FROM amazon_reviews
GROUP BY year, product_title
QUALIFY ROW_NUMBER() OVER (PARTITION BY year ORDER BY COUNT(*) DESC) = 1;

-- Task 5.8: Users Who Wrote the Most Reviews
SELECT customer_id, COUNT(*) AS review_count
FROM amazon_reviews
GROUP BY customer_id
ORDER BY review_count DESC
LIMIT 10;


In [ ]:
CREATE INDEX idx_review_date ON amazon_reviews(review_date);
CREATE INDEX idx_product_title ON amazon_reviews(product_title);
CREATE INDEX idx_customer_id ON amazon_reviews(customer_id);
